In [83]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(fgsea)
library(data.table)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')

In [84]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv")

In [85]:
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)


[1] "Total reading time: 15.9849999999997 seconds"
[1] "The length of the list matches the length of the input path."


In [86]:
background_gene<-read.csv('filtered_gene_Y2020-2021_Y2021-2022_D7.csv') %>% filter(AIFI_L3=='CD27- effector B cell')

In [87]:
gene_set<-read.csv('gsea_result.csv') %>% filter(pathway=='HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY',celltype=='CD27- effector B cell')
gene_vector <- eval(parse(text = gene_set$leadingEdge))

In [88]:
  
celltype_list <- lapply(df_list, function(df) { df <- df[background_gene$gene, grep('CD27- effector B cell', names(df), fixed = TRUE),drop=FALSE] })


In [89]:
exp_matrix<-do.call(cbind,celltype_list)

colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data)<-meta_data$pbmc_sample_id

In [92]:
# SLEA - assigning pathway scores
doSLEA <- function(expressionSet, geneSet) {
  # scale expression
  exprsMat <- expressionSet
  # exprsMat <- t(scale(t(exprsMat)))
  # extract expression of leGenes of each geneset
  comm <- intersect(geneSet, rownames(expressionSet))
  gsDF <- exprsMat[comm, ]
  # calculate mean expression per sample
  gsM <- colMeans(gsDF)
  # extract random genes of size of the geneSet from full probeset and calculate mean
  # and perform this for 'n' permutations
  nperm <- lapply(1:1000, function(j) {
    # set seed for every permutation
    set.seed(j)
    rGSDF <- exprsMat[sample.int(nrow(exprsMat),length(comm)), ]
    rGSM <- colMeans(rGSDF)
    return(value = rGSM)
  })
  permDF <- do.call(rbind, nperm)
  zscore <- (gsM - colMeans(permDF)) / apply(permDF,2,sd)
  sleaDF <- zscore %>% as.data.frame()
  return(value = sleaDF)
}

In [93]:
res<-doSLEA(exp_matrix,gene_vector)

In [94]:
colnames(res)<-"pathway_score"

In [95]:
res$pbmc_sample_id<-rownames(res)
res<-left_join(res,meta_data)

Joining with `by = join_by(pbmc_sample_id)`


In [96]:
res_summary<-res %>% group_by(Flu_Year,Flu_Day,cohort.cohortGuid) %>% summarise(mean_pathway_score=mean(pathway_score) )

`summarise()` has grouped output by 'Flu_Year', 'Flu_Day'. You can override
using the `.groups` argument.


In [97]:
res_summary<-res_summary %>% arrange(cohort.cohortGuid,Flu_Year)

In [98]:
write.csv(res_summary,"SLEA_CD27neg_Effector_B.csv")